<a href="https://colab.research.google.com/github/GurashishJuneja/MMAI-891-NLP-Project/blob/dev/Tony/XLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1.Initial import of packages and mount**


In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, XLMRobertaXLModel
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

import os,re
from sklearn.model_selection import train_test_split



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# mount Google Drive to Google Colab notebook
from google.colab import drive
drive.mount('/content/NewDrive')

Drive already mounted at /content/NewDrive; to attempt to forcibly remount, call drive.mount("/content/NewDrive", force_remount=True).


In [ ]:
train_df = pd.read_csv("/content/NewDrive/MyDrive/MMAI 891 NLP Project/Train.csv")
test_df = pd.read_csv("/content/NewDrive/MyDrive/MMAI 891 NLP Project/Test.csv")
train_df.info()
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001 entries, 0 to 10000
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   tweet_id   10001 non-null  object 
 1   safe_text  10001 non-null  object 
 2   label      10000 non-null  float64
 3   agreement  9999 non-null   float64
dtypes: float64(2), object(2)
memory usage: 312.7+ KB


In [ ]:
# define valid label values
valid_labels = [-1, 0, 1]

# filter out rows with invalid label values
train_df = train_df[train_df['label'].isin(valid_labels)]

# reset the index after filtering
train_df = train_df.reset_index(drop=True)

In [ ]:
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

In [ ]:
# Load data from train dataframe
train_text = train_df['safe_text']
val_text = val_df['safe_text']

# **Preprocessing**

In [ ]:
def tokenize_sentences(sentences):
    input_ids = []
    attention_masks = []

    for sent in sentences:
        encoded_dict = tokenizer.encode_plus(
                            sent,
                            add_special_tokens = True,
                            max_length = 40,
                            padding = 'max_length',
                            truncation=True,
                            return_attention_mask = True,
                            return_tensors = 'pt'
                       )       
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
    
    return torch.cat(input_ids, dim=0), torch.cat(attention_masks, dim=0)
    
input_ids, attention_masks = tokenize_sentences(train_df['safe_text'])

In [ ]:
pattern = r'\(\@.*?\)'
train_text = [re.sub(pattern, '', text) for text in train_text]
val_text = [re.sub(pattern, '', text) for text in val_text]


In [ ]:
# Text normalization
def normalize_text(text):
    text = text.lower()
    text = re.sub(r'http\S+', '', text) # remove urls
    text = re.sub(r'@\w+', '', text) # remove @users
    text = re.sub(r'#\w+', '', text) # remove hashtags
    text = re.sub(r'[^\w\s#@/:%.,_-]+', '', text) # remove emoticons
    text = re.sub(r'\d+', '', text) # remove numbers
    text = re.sub(r'[^\w\s]', '', text) # remove punctuation
    text = re.sub(r'\s+', ' ', text) # remove extra whitespace
    return text

normalized_train = [normalize_text(t) for t in train_text]
normalized_val = [normalize_text(t) for t in val_text]


In [ ]:
unwanted_words = ['url', 'user']
def remove_unwanted_words(sentences, unwanted_words):
    result = []
    for sentence in sentences:
        words = sentence.split()
        filtered_words = [word for word in words if word.lower() not in unwanted_words]
        filtered_sentence = " ".join(filtered_words)
        result.append(filtered_sentence)
    return result

cleaned_train = remove_unwanted_words(normalized_train, unwanted_words)
cleaned_val = remove_unwanted_words(normalized_val, unwanted_words)

In [ ]:
def map_labels(label):
    if label == -1:
        return 0
    elif label == 0:
        return 1
    else:
        return 2

In [ ]:
# preprocess train set
max_length = 32
train_label = train_df['label'].tolist()
encoded_inputs_train = tokenizer(cleaned_train, padding=True, truncation=True, max_length=max_length, return_tensors='pt')
train_input_ids = encoded_inputs_train['input_ids']
train_attention_mask = encoded_inputs_train['attention_mask']
train_labels = [map_labels(label) for label in train_label]
train_labels = torch.tensor(train_labels, dtype=torch.long)



In [ ]:
# preprocess validation set
val_label = val_df['label'].tolist()
encoded_inputs_val = tokenizer(cleaned_val, padding=True, truncation=True, max_length=max_length, return_tensors='pt')
val_input_ids = encoded_inputs_val['input_ids']
val_attention_mask = encoded_inputs_val['attention_mask']
val_labels = [map_labels(label) for label in val_label]
val_labels = torch.tensor(val_labels, dtype=torch.long)

In [ ]:
import numpy as np

# assuming you have a tensor object called 'tensor_obj'
np.unique(val_labels.numpy(), return_counts=True)

(array([0, 1, 2]), array([240, 943, 817]))

# XLM build up

In [ ]:
train_dataset = TensorDataset(train_input_ids, train_attention_mask, train_labels)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

val_dataset = TensorDataset(val_input_ids, val_attention_mask, val_labels)
val_loader = DataLoader(val_dataset, batch_size=64)


# # preprocess tweets
# max_length = 64
# batch_size = 128
# num_samples = len(train_df)
# num_batches = (num_samples + batch_size - 1) // batch_size

classifier = nn.Sequential(
    nn.Linear(model.config.hidden_size, 3)
)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=1e-5)

In [ ]:
# set up checkpoint saving
checkpoint_path = '/content/NewDrive/MyDrive/MMAI 891 NLP Project/XLMcheckpoint.pth'
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    classifier.load_state_dict(checkpoint['classifier_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch_start = checkpoint['epoch'] + 1
else:
    epoch_start = 1

In [ ]:
for epoch in range(3):
    epoch_loss = 0.0

    # train on train set
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        logits = model(input_ids, attention_mask=attention_mask).last_hidden_state[:, 0, :]
        preds = classifier(logits)
        loss = criterion(preds, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # evaluate on validation set
    correct = 0
    total = 0
    for batch in val_loader:
        input_ids, attention_mask, labels = batch
        logits = model(input_ids, attention_mask=attention_mask).last_hidden_state[:, 0, :]
        preds = classifier(logits)
        _, predicted = torch.max(preds.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    acc = correct / total
    print(f'Epoch {epoch + 1}, Loss: {loss.item()}, Validation Accuracy: {acc}')

      # save checkpoint
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'classifier_state_dict': classifier.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }
    torch.save(checkpoint, checkpoint_path)

Epoch 1, Loss: 1.0038564205169678, Validation Accuracy: 0.4725
Epoch 2, Loss: 1.0283615589141846, Validation Accuracy: 0.496
Epoch 3, Loss: 0.9423801302909851, Validation Accuracy: 0.5115


# Load test dataframe

In [ ]:
test_df = pd.read_csv("/content/NewDrive/MyDrive/MMAI 891 NLP Project/Test.csv")
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5177 entries, 0 to 5176
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet_id   5177 non-null   object
 1   safe_text  5176 non-null   object
dtypes: object(2)
memory usage: 81.0+ KB


In [ ]:
test_df = test_df.dropna(subset=['safe_text'])
test_text = train_df['safe_text']
test_text = [re.sub(pattern, '', text) for text in test_text]
normalized_test = [normalize_text(t) for t in test_text]
cleaned_test = remove_unwanted_words(normalized_test, unwanted_words)


In [ ]:
type_counts = {}
for element in cleaned_test:
    element_type = type(element).__name__
    if element_type in type_counts:
        type_counts[element_type] += 1
    else:
        type_counts[element_type] = 1

for element_type, count in type_counts.items():
    print(f"{element_type}: {count}")

str: 7999


In [ ]:
float_elements = []
for element in cleaned_test:
    if isinstance(element, float):
        float_elements.append(element)

print(float_elements)

[]


In [ ]:
# set batch size and initialize empty list for predictions
batch_size = 16
test_preds = []

# loop over batches of test data
for i in range(0, len(test_df), batch_size):
    # preprocess batch of test tweets and get logits
    batch_encoded_inputs = tokenizer(cleaned_test, padding=True, truncation=True, max_length=max_length, return_tensors='pt')
    batch_input_ids = batch_encoded_inputs['input_ids']
    batch_attention_mask = batch_encoded_inputs['attention_mask']
    batch_outputs = model(batch_input_ids, attention_mask=batch_attention_mask)
    batch_logits = batch_outputs.last_hidden_state[:, 0, :]

    # get batch predictions and add to list
    batch_preds = classifier(batch_logits).argmax(dim=1).tolist()
    test_preds += batch_preds

# save predictions to file
output_df = pd.DataFrame({'ID': test_df['tweet_id'], 'target': test_preds})
output_df.to_csv('/content/NewDrive/MyDrive/MMAI 891 NLP Project/predictions.csv', index=False)


In [ ]:
output_df.info()

In [ ]:
value_counts = output_df['target'].value_counts()
print(value_counts)